In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def conv_block(x, filters, name=None):
    x = layers.Conv2D(
        filters, 3, padding="same", activation="relu", name=f"{name}_conv1"
    )(x)
    x = layers.BatchNormalization(name=f"{name}_bn1")(x)

    x = layers.Conv2D(
        filters, 3, padding="same", activation="relu", name=f"{name}_conv2"
    )(x)
    x = layers.BatchNormalization(name=f"{name}_bn2")(x)

    return x

In [3]:
def encoder_block(x, filters, name):
    x = conv_block(x, filters, name=name)
    skip = x                      # lưu cho skip connection
    x = layers.MaxPooling2D((2, 2), name=f"{name}_pool")(x)
    return x, skip

In [4]:
def decoder_block(x, skip, filters, name):
    x = layers.Conv2DTranspose(
        filters, (2, 2), strides=2, padding="same", name=f"{name}_up"
    )(x)

    x = layers.Concatenate(name=f"{name}_concat")([x, skip])
    x = conv_block(x, filters, name=name)

    return x

In [8]:
def build_unet_denoise(input_shape=(256, 256, 1)):
    inputs = layers.Input(shape=input_shape, name="noisy_spectrogram")

    # -------- Encoder --------
    x1, s1 = encoder_block(inputs, 64, "enc1")
    x2, s2 = encoder_block(x1, 128, "enc2")
    x3, s3 = encoder_block(x2, 256, "enc3")
    x4, s4 = encoder_block(x3, 512, "enc4")

    # -------- Bottleneck --------
    b = conv_block(x4, 1024, "bottleneck")

    # -------- Decoder --------
    d4 = decoder_block(b, s4, 512, "dec4")
    d3 = decoder_block(d4, s3, 256, "dec3")
    d2 = decoder_block(d3, s2, 128, "dec2")
    d1 = decoder_block(d2, s1, 64, "dec1")

    # -------- Output (mask) --------
    mask = layers.Conv2D(
        1, (1, 1), activation="sigmoid", name="mask"
    )(d1)

    model = models.Model(inputs, mask, name="UNet_Denoise")
    return model

In [9]:
model = build_unet_denoise()
model.summary()

Model: "UNet_Denoise"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ noisy_spectrogram   │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc1_conv1 (Conv2D) │ (None, 256, 256,  │        640 │ noisy_spectrogra… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc1_bn1            │ (None, 256, 256,  │        256 │ enc1_conv1[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc1_conv2 (Conv2D) │ (None, 256, 256,  │     36,928 │ enc1_bn1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc1_bn2            │ (None, 256, 256,  │        256 │ enc1_conv2[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc1_pool           │ (None, 128, 128,  │          0 │ enc1_bn2[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc2_conv1 (Conv2D) │ (None, 128, 128,  │     73,856 │ enc1_pool[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc2_bn1            │ (None, 128, 128,  │        512 │ enc2_conv1[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc2_conv2 (Conv2D) │ (None, 128, 128,  │    147,584 │ enc2_bn1[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc2_bn2            │ (None, 128, 128,  │        512 │ enc2_conv2[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc2_pool           │ (None, 64, 64,    │          0 │ enc2_bn2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc3_conv1 (Conv2D) │ (None, 64, 64,    │    295,168 │ enc2_pool[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc3_bn1            │ (None, 64, 64,    │      1,024 │ enc3_conv1[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc3_conv2 (Conv2D) │ (None, 64, 64,    │    590,080 │ enc3_bn1[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc3_bn2            │ (None, 64, 64,    │      1,024 │ enc3_conv2[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc3_pool           │ (None, 32, 32,    │          0 │ enc3_bn2[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc4_conv1 (Conv2D) │ (None, 32, 32,    │  1,180,160 │ enc3_pool[0][0] 

 Total params: 31,054,145 (118.46 MB)

 Trainable params: 31,042,369 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)